In [ ]:
import requests
from bs4 import BeautifulSoup

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}


url_list = []

for i in range(2):
    url = 'https://www.metacritic.com/browse/movie/?releaseYearMin=1910&releaseYearMax=2023&page=' + str(i)
    url_list.append(url)
url_list

movie_1000 = []


for i in url_list:
    html = requests.get(i, headers = headers)
    soup = BeautifulSoup(html.content, "html.parser")
    hyperlink_list = soup.find_all('a', class_ = "c-finderProductCard_container g-color-gray80 u-grid")
    for i in hyperlink_list:
        movie_page = 'https://www.metacritic.com' + i['href']
        movie_1000.append(movie_page)

print(len(movie_1000))
print(movie_1000)

dict_1000 = {}
n = 0

for i in movie_1000:
    # getting all the 3 information from each movie
    n = n + 1 
    print(n)
    movie_page = requests.get(i, headers=headers)
    movie_page_soup = BeautifulSoup(movie_page.content, "html.parser") 
    movie_name_list = movie_page_soup.find_all('div', class_="c-productHero_title g-inner-spacing-bottom-medium g-outer-spacing-top-medium")

    for movie_name_tag in movie_name_list:
        movie_name = movie_name_tag.get_text().strip()

        view_all = movie_page_soup.find_all('a', class_="c-globalHeader_viewMore g-color-gray50 g-text-xxsmall u-text-uppercase")

        for view_all_link_tag in view_all:
            view_all_link = 'https://www.metacritic.com' + view_all_link_tag['href']

        view_all_page = requests.get(view_all_link, headers=headers)
        view_all_soup = BeautifulSoup(view_all_page.content, "html.parser")
        credit_tables_list = view_all_soup.find_all('div', class_="c-productCredits g-outer-spacing-bottom-xlarge")

        directors_dict = {}  # Initialize a dictionary for directors and cast
        casts = []

        for credit_table in credit_tables_list:
            if "Directed By" in credit_table.get_text().strip():
                director_names = [director_tag.get_text().strip() 
                for director_tag in credit_table.find_all('a')]
                
                for director_name in director_names:
                    directors_dict[director_name] = {}
                    
            table_title = credit_table.find('h3')
            if table_title and "cast" in table_title.get_text().strip().lower():
                cast_names = [j.get_text().strip() 
                for j in credit_table.find_all('a')]
                casts.extend(cast_names)
        
        sub_dict = {}
        sub_dict[tuple(directors_dict.keys())] = casts
        dict_1000[movie_name] = sub_dict
print(dict_1000)

# Writing to CSV file
import csv
csv_file_path = 'movie_info7.csv'

with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Movie Name', 'Directors', 'Cast']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    for movie_name, sub_dict in dict_1000.items():
        for directors, cast in sub_dict.items():
            writer.writerow({
                'Movie Name': movie_name,
                'Directors': ', '.join(directors),
                'Cast': ', '.join(cast)
            })

print(f"Data has been written to {csv_file_path}")
print("What do you want to check on Metacrtitics? (Please choose ‘movie’, ‘people’, or ‘comparison’)")
choice = input('movie’ or ‘people’ or comparison’?')

if choice == 'movie':
    choice2 = input('Pick movie name: ')
    
    for movie_name, director_dict in dict_1000.items():
        for director_name_tuple, cast in director_dict.items():
            director_name = director_name_tuple[0]

            if movie_name.lower() == choice2.lower():
                print('The director is', director_name)
                print('The cast members are', ', '.join(cast))
                for cast_member in cast:
                    print(f'- {cast_member} as {input(f"Character played by {cast_member}: ")}')
                
if choice == 'people':
    choice2 = input('Pick director: ')
    movie_list = []
    cast_list = []

    for movie_name, director_dict in dict_1000.items():
        for director_name_tuple, cast in director_dict.items():
            director_name = director_name_tuple[0]

            if choice2.lower() in director_name.lower():
                movie_list.append(movie_name)
                cast_list.extend(cast)

    print('Movies directed by', choice2, ':', ', '.join(movie_list))
    print('Cast members across movies:', ', '.join(cast_list))



import math

if choice == 'comparison':
    choice1 = input('Pick 1st director: ')
    choice2 = input('Pick 2nd director: ')
    
    movie_list1 = []
    cast_list1 = []
    director_cast_count1 = {}  
    
    movie_list2 = []
    cast_list2 = []
    director_cast_count2 = {}  

    for movie_name, director_dict in dict_1000.items():
        for director_name_tuple, cast in director_dict.items():
            director_name = director_name_tuple[0]

            if choice1.lower() in director_name.lower():
                movie_list1.append(movie_name)
                cast_list1.extend(cast)
                for cast_member in cast:
                    director_cast_count1[cast_member] = director_cast_count1.get(cast_member, 0) + 1
                   # print(f'{cast_member}: {director_cast_count1[cast_member]}')


            if choice2.lower() in director_name.lower():
                movie_list2.append(movie_name)
                cast_list2.extend(cast)
                for cast_member in cast:
                    director_cast_count2[cast_member] = director_cast_count2.get(cast_member, 0) + 1

    print("\n")
    print(f'Movies directed by {choice1}: {", ".join(movie_list1)}')
    print(f'Cast members across movies ({choice1}): {", ".join(f"{member} ({count})" for member, count in director_cast_count1.items())}')
    
    print(f'\nMovies directed by {choice2}: {", ".join(movie_list2)}')
    print(f'Cast members across movies ({choice2}): {", ".join(f"{member} ({count})" for member, count in director_cast_count2.items())}')

    # Calculate Cosine Similarity
    common_casts = set(director_cast_count1.keys()).intersection(director_cast_count2.keys())

    dot_product = sum(director_cast_count1[cast] * director_cast_count2[cast] for cast in common_casts)
    magnitude1 = math.sqrt(sum(count**2 for count in director_cast_count1.values()))
    magnitude2 = math.sqrt(sum(count**2 for count in director_cast_count2.values()))

    cosine_similarity = dot_product / (magnitude1 * magnitude2)

    print(f'Cosine similarity between casts of {choice1} and {choice2}: {cosine_similarity}')
    print(f'Common cast members: {", ".join(common_casts)}')
